In [1]:
"""
A class that wraps the the GO ontology that constructs a graph from the GO OBO file.
It provides methods to construct the GO graph, compute various GO-related metrics,
propagate terms, and plot GO subgraphs in various ways.
"""

from pathlib import Path
from typing import Optional, Union, Tuple, Container, List, Callable

import networkx as nx
from networkx.drawing import layout
from matplotlib import pyplot as plt
from goatools.obo_parser import GODag


MF_ROOT_ID = 'GO:0003674'
CC_ROOT_ID = 'GO:0005575'
BP_ROOT_ID = 'GO:0008150'


class GOGraph:
    """
    A wrapper for the the GO ontology that constructs a graph from the GO OBO file.
    It provides methods to construct the GO graph, compute various GO-related metrics,
    propagate terms, and plot GO subgraphs in various ways.

    Args:
        go_path: The path to the GO OBO file.
        root_ids: The root IDs of the GO graph.
        discard_go_dag: Whether to discard the GOATools GODag object after construction.
            If True, the GODag object will not be available after construction.
            If False, the GODag object will be available as self.go_dag.
    """
    def __init__(self, go_fpath: Union[Path, str],
                 root_ids: Optional[Tuple[str, str, str]] = (MF_ROOT_ID, CC_ROOT_ID, BP_ROOT_ID),
                 discard_go_dag: bool = False):
        self.go_fpath = go_fpath
        go_dag = GODag(go_fpath)
        self.ids_to_terms = {go_id: term_node.name for go_id, term_node in go_dag.items()}
        self.root_ids = root_ids
        self.go_graph = self.construct_graph_from_godag(go_dag)
        if not discard_go_dag:
            self.go_dag = go_dag

    def construct_graph_from_godag(self,
        godag: GODag,
        max_depth: Optional[int] = None,
        direction: str = 'both'
    ) -> nx.DiGraph:
        """
        Construct a NetworkX directed graph from a GOATools GODag object.
        
        Args:
            godag: A GOATools GODag object containing the Gene Ontology hierarchy
            root_node: Optional root node ID to start subgraph construction from
            max_depth: Optional maximum depth to traverse from root node
            direction: Direction to traverse from root ('up', 'down', or 'both')
            
        Returns:
            A NetworkX directed graph with both 'PARENT' and 'CHILD' edges
        """
        G = nx.DiGraph()
        queue = []
        visited = set()

        def add_node_and_edges(go_id: str, current_depth: int = 0):
            """Helper function to add nodes and edges"""
            if go_id not in godag or go_id in visited:
                return

            term = godag[go_id]
            # print(term.id, term.name)
            visited.add(go_id)

            # Add the node if not already present
            # print('Checking if node exists', go_id, go_id in G)
            if go_id not in G:
                # print('Adding node', go_id)
                G.add_node(go_id,
                        name=term.name,
                        namespace=term.namespace,
                        level=term.level,
                        depth=term.depth)

            # If we've reached max depth, stop traversing
            if max_depth is not None and current_depth >= max_depth:
                return

            # Add parent relationships (traverse up)
            if direction in ['up', 'both']:
                for parent in term.parents:
                    if parent.id not in visited:
                        queue.append((parent.id, current_depth + 1))
                        G.add_node(go_id,
                                name=parent.name,
                                namespace=parent.namespace,
                                level=parent.level,
                                depth=parent.depth)
                    # print('Adding edge', go_id, parent.id)
                    G.add_edge(go_id, parent.id, type='PARENT')
                    G.add_edge(parent.id, go_id, type='CHILD')

            # Add child relationships (traverse down)
            if direction in ['down', 'both']:
                for child in term.children:
                    if child.id not in visited:
                        queue.append((child.id, current_depth + 1))
                        G.add_node(child.id,
                                name=child.name,
                                namespace=child.namespace,
                                level=child.level,
                                depth=child.depth)
                    G.add_edge(go_id, child.id, type='CHILD')
                    G.add_edge(child.id, go_id, type='PARENT')

        for go_id in self.root_ids:
            queue.append((go_id, 0))

        while queue:
            go_id, current_depth = queue.pop(0)
            add_node_and_edges(go_id, current_depth)

        return G

    def compute_excess_components(self,
                                  predicted_terms: Container[str],
                                  expected_num_components: Union[int, None]=None) -> int:
        """
        Computes the number of excess disconnected components in the predicted subgraph.

        Args:
            predicted_terms: A container of GO IDs representing the predicted terms.
            expected_num_components: The expected number of components. If None the expected number of components 
                is the number of root IDs used to construct the graph.
        
        Returns:
            The number of excess disconnected components in the predicted subgraph.
        """
        if expected_num_components is None:
            expected_num_components = len(self.root_ids)
        predicted_subgraph = self.go_graph.subgraph(predicted_terms)
        count_weak = nx.number_weakly_connected_components(predicted_subgraph)
        return max(0, count_weak - expected_num_components)

    def compute_excess_components_per_term(self,
                                           predicted_terms: Container[str],
                                           expected_num_components: Union[int, None]=None) -> float:
        """
        Computes the number of excess disconnected components in the predicted subgraph normalized by the number of terms.

        Args:
            predicted_terms: A container of GO IDs representing the predicted terms.
            expected_num_components: The expected number of components. If None the expected number of components 
                is the number of root IDs used to construct the graph.

        Returns:
            The number of excess disconnected components in the predicted subgraph normalized by the number of terms.
        """
        return self.compute_excess_components(predicted_terms=predicted_terms,
                                            expected_num_components=expected_num_components) / len(predicted_terms)

    @staticmethod
    def compute_depths(G: nx.DiGraph, root_id: str) -> List[int]:
        """
        Returns a list of depth values for each node in the graph.
        The depth of a node is the number of edges from the root to the node as found by breadth-first search 
            (this may be different from the "depth" attribute of the nodes).
        The root is the node with the given root_id.

        Args:
            G: A NetworkX directed graph.
            root_id: The ID of the root node to start the breadth-first search from.

        Returns:
            A list of depth values for each node in the graph.
        """
        depths = []
        queue = [(root_id, 0)]
        while queue:
            node_id, depth = queue.pop(0)
            depths.append(depth)
            for _, dest, data in G.edges(node_id, data=True):
                if data['type'] == 'CHILD':
                    queue.append((dest, depth + 1))
        return depths

    def propagate_terms(self, source_terms: Container[str], direction: str='up') -> List[str]:
        """
        Returns a list of node ids representing the subgraph induced by following edges from the source terms.
        If the direction is 'up', only 'PARENT' edges are followed.
        If the direction is 'down', only 'CHILD' edges are followed.
        If the direction is 'both', both 'PARENT' and 'CHILD' edges are followed.

        Args:
            source_terms: A container of GO IDs representing the source terms.
            direction: The direction to propagate the terms. Allowed values are "up" and "down" and "both"

        Returns:
            A list of GO IDs representing the propagated terms.
        """
        if direction == 'up':
            edge_types = ('PARENT',)
        elif direction == 'down':
            edge_types = ('CHILD',)
        elif direction == 'both':
            edge_types = ('PARENT', 'CHILD')
        else:
            raise ValueError(f'Invalid direction: {direction}. Allowed directions are "up", "down", and "both".')

        propagated_nodes = []

        for term in source_terms:
            parent_queue = [term]

            while parent_queue:
                this_node = parent_queue.pop()
                propagated_nodes.append(this_node)
                for dest_node, edge_data in self.go_graph[this_node].items():
                    if edge_data['type'] in edge_types:
                        parent_queue.append(dest_node)
        return propagated_nodes

    @staticmethod
    def plot_go_subgraph(G: nx.DiGraph,
                           ax: Optional[plt.Axes]=None,
                           label_by_name: bool=False,
                           layout_func: Callable=layout.spring_layout,
                           fig_width: float=12,
                           node_size: float=50,
                           edge_width: float=0.5,
                           node_color: str='lightblue',
                           edge_color: str='gray',
                           label_size: float=6,
                           ) -> Tuple[plt.Figure, plt.Axes]:
        '''
        Plots a subgraph of G containing only the nodes in node_list and only 'CHILD' edges.
        '''
        pos = layout_func(G)
        if ax is None:
            fig, ax = plt.subplots(1, figsize=(fig_width, fig_width), dpi=300)
        else:
            fig = ax.get_figure()

        nx.draw_networkx_nodes(
            G=G,
            pos=pos,
            ax=ax,
            node_color=node_color,
            node_size=node_size
        )
        # Filter edges to only those labeled as 'CHILD'
        child_edges = [
            (u, v) for u, v, d in G.edges(data=True)
            if d.get('type') == 'CHILD'
        ]
        nx.draw_networkx_edges(
            G=G,
            pos=pos,
            edgelist=child_edges,
            ax=ax,
            edge_color=edge_color,
            arrows=True,
            width=edge_width
        )
        if label_by_name:
            node_labels = {node: G.nodes[node]['name'] for node in G.nodes()}
        else:
            node_labels = {node: node for node in G.nodes()}
        nx.draw_networkx_labels(
            G=G,
            pos=pos,
            ax=ax,
            labels=node_labels,
            verticalalignment='bottom',
            horizontalalignment='center',
            font_size=label_size
        )
        ax.set_axis_off()
        return fig, ax

    def plot_go_subgraphs_true_vs_predicted(self,
                                            true_terms: Container[str],
                                            predicted_terms: Container[str],
                                            ax: Optional[plt.Axes]=None,
                                            label_by_name: bool=False,
                                            layout_func: Callable=layout.spring_layout,
                                            fig_width: float=8,
                                            node_size: float=50,
                                            edge_width: float=0.5,
                                            label_size: float=6,
                                            tp_color: str='green',
                                            fp_color: str='red',
                                            fn_color: str='blue',
                                            edge_color: str='gray',
                                            ) -> Tuple[plt.Figure, plt.Axes]:
        '''
        Plots a subgraph of the GO graph containing the nodes representing the true and predicted terms.
        The nodes are colored green, red, and blue (default colors) for true positives, false positives,
            and false negatives, respectively.
        The edges are colored gray (default). Only the 'CHILD' edges are shown.
        The nodes are labeled by their GO IDs, or optionally by the term names.
        
        Args:
            true_terms: A container of GO IDs representing the true terms.
            predicted_terms: A container of GO IDs representing the predicted terms.
            ax: An optional matplotlib Axes object to plot the graph on.
            label_by_name: Whether to label the nodes by the term names instead of their GO IDs.
            layout_func: A function to layout the graph.
            fig_width: The width of the figure.
            node_size: The size of the nodes.
            edge_width: The width of the edges.
            label_size: The size of the labels.
            
        '''
        true_terms = set(true_terms)
        predicted_terms = set(predicted_terms)
        tp_terms = true_terms.intersection(predicted_terms)
        fp_terms = predicted_terms.difference(tp_terms)
        fn_terms = true_terms.difference(tp_terms)
        joint_terms = true_terms.union(predicted_terms)

        joint_network = self.go_graph.subgraph(joint_terms)

        pos = layout_func(joint_network)

        if ax is None:
            fig, ax = plt.subplots(1, figsize=(fig_width, fig_width), dpi=300)
        else:
            fig = ax.get_figure()

        nx.draw_networkx_nodes(
            G=self.go_graph.subgraph(tp_terms),
            pos=pos,
            ax=ax,
            node_color=tp_color,
            node_size=node_size
        )

        nx.draw_networkx_nodes(
            G=self.go_graph.subgraph(fp_terms),
            pos=pos,
            ax=ax,
            node_color=fp_color,
            node_size=node_size
        )

        nx.draw_networkx_nodes(
            G=self.go_graph.subgraph(fn_terms),
            pos=pos,
            ax=ax,
            node_color=fn_color,
            node_size=node_size
        )

        # Filter edges to only those labeled as 'CHILD'
        child_edges = [
            (u, v) for u, v, d in joint_network.edges(data=True)
            if d.get('type') == 'CHILD'
        ]
        nx.draw_networkx_edges(
            G=joint_network,
            pos=pos,
            edgelist=child_edges,
            ax=ax,
            edge_color=edge_color,
            arrows=True,
            width=edge_width
        )
        if label_by_name:
            node_labels = {node: joint_network.nodes[node]['name'] for node in joint_network.nodes()}
        else:
            node_labels = {node: node for node in joint_network.nodes()}
        nx.draw_networkx_labels(
            G=joint_network,
            pos=pos,
            ax=ax,
            labels=node_labels,
            verticalalignment='bottom',
            horizontalalignment='center',
            font_size=label_size
        )
        ax.set_axis_off()
        return fig, ax


In [6]:
cafa5_go_path = '../cafa-5-protein-function-prediction/Train/go-basic.obo'
godag = GODag(cafa5_go_path)


../cafa-5-protein-function-prediction/Train/go-basic.obo: fmt(1.2) rel(2023-01-01) 46,739 Terms


In [5]:
dir(godag)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_populate_relationships',
 '_populate_terms',
 '_set_level_depth',
 '_str_desc',
 'clear',
 'copy',
 'data_version',
 'draw_lineage',
 'fromkeys',
 'get',
 'id2int',
 'items',
 'keys',
 'label_wrap',
 'load_obo_file',
 'make_graph_pydot',
 'make_graph_pygraphviz',
 'paths_to_top',
 'pop',
 'popitem',
 'query_term',
 'setdefault',
 'typedefs',
 'update',
 'update_association',
 'values',
 'version',

In [33]:
with open('../cafa-5-protein-function-prediction/Train/go-basic.obo', 'r') as f:
    obo_contents = f.read()

In [34]:
print(obo_contents[:100000])

format-version: 1.2
data-version: releases/2023-01-01
subsetdef: chebi_ph7_3 "Rhea list of ChEBI terms representing the major species at pH 7.3."
subsetdef: gocheck_do_not_annotate "Term not to be used for direct annotation"
subsetdef: gocheck_do_not_manually_annotate "Term not to be used for direct manual annotation"
subsetdef: goslim_agr "AGR slim"
subsetdef: goslim_aspergillus "Aspergillus GO slim"
subsetdef: goslim_candida "Candida GO slim"
subsetdef: goslim_chembl "ChEMBL protein targets summary"
subsetdef: goslim_drosophila "Drosophila GO slim"
subsetdef: goslim_flybase_ribbon "FlyBase Drosophila GO ribbon slim"
subsetdef: goslim_generic "Generic GO slim"
subsetdef: goslim_metagenomics "Metagenomics GO slim"
subsetdef: goslim_mouse "Mouse GO slim"
subsetdef: goslim_pir "PIR GO slim"
subsetdef: goslim_plant "Plant GO slim"
subsetdef: goslim_pombe "Fission yeast GO slim"
subsetdef: goslim_synapse "synapse GO slim"
subsetdef: goslim_yeast "Yeast GO slim"
subsetdef: prokaryote_subset

In [75]:
from typing import Iterable

GO_BASIC_MULTIVALUED_KEYS = ['is_a', 'relationship', 'synonym', 'xref', 'alt_id', 'subset', 'consider']


def parse_obo_content(obo_lines: Iterable[str], multivalued_keys: Container[str]=GO_BASIC_MULTIVALUED_KEYS) -> dict:
    """
    Parse an OBO file and return a dictionary of header keys and values, and a dictionary of atom types and their values.
    Currently defined "atoms" in the go-basic.obo file are: "Term" and "Typedef". The Typedef entries define relationship types between Terms
    """
    def parse_kv(line: str) -> tuple[str, str]:
        key, value = line.split(':', 1)
        key = key.strip()
        value = value.strip()
        return key, value
    
    multivalued_keys = set(multivalued_keys)

    header = {}
    content = {}
    current_atom = {}
    current_atom_type = None
    for line_num, line in enumerate(obo_lines):
        line = line.strip()
        # print(line_num, line)
        if not line:
            continue
        if line[0] == '[' and line[-1] == ']':
            if current_atom:
                # print(f"Adding new atom of type: {current_atom_type}, {current_atom}")
                content[current_atom_type].append(current_atom)
                current_atom = {}
            current_atom_type = line[1:-1]
            if current_atom_type not in content:
                # print(f"Adding new atom type: {current_atom_type}")
                content[current_atom_type] = []

            continue
        try:
            key, value = parse_kv(line)
        except ValueError:
            # print(f"Error parsing line {line_num}: {line}")
            continue
        if current_atom_type is None:
            # print(f"Adding header: {key}, {value}")
            header[key] = value
        else:
            if key in multivalued_keys:
                if key not in current_atom:
                    current_atom[key] = []
                current_atom[key].append(value)
            else:
                current_atom[key] = value
            
    if current_atom:
        # print(f"Adding new atom of type: {current_atom_type}, {current_atom}")
        content[current_atom_type].append(current_atom)

    return header, content


with open('../cafa-5-protein-function-prediction/Train/go-basic.obo', 'r') as f:
    header, content = parse_obo_content(f)

print(header)
for key in content:
    print(key, len(content[key]))

{'format-version': '1.2', 'data-version': 'releases/2023-01-01', 'subsetdef': 'prokaryote_subset "GO subset for prokaryotes"', 'synonymtypedef': 'systematic_synonym "Systematic synonym" EXACT', 'default-namespace': 'gene_ontology', 'ontology': 'go'}
Term 47417
Typedef 5


In [71]:
max_values = {}
for term in content['Term']:
    for key, values in term.items():
        if key not in max_values:
            max_values[key] = 0
        max_values[key] = max(max_values[key], len(values))

max_values

{'id': 1,
 'name': 1,
 'namespace': 1,
 'def': 1,
 'synonym': 284,
 'is_a': 11,
 'alt_id': 16,
 'subset': 12,
 'xref': 465,
 'comment': 1,
 'is_obsolete': 1,
 'consider': 8,
 'relationship': 3,
 'replaced_by': 1}

In [76]:
content['Term'][5:10]

[{'id': 'GO:0000007',
  'name': 'low-affinity zinc ion transmembrane transporter activity',
  'namespace': 'molecular_function',
  'def': '"Enables the transfer of a solute or solutes from one side of a membrane to the other according to the reaction: Zn2+ = Zn2+, probably powered by proton motive force. In low-affinity transport the transporter is able to bind the solute only if it is present at very high concentrations." [GOC:mtg_transport, ISBN:0815340729]',
  'is_a': ['GO:0005385 ! zinc ion transmembrane transporter activity']},
 {'id': 'GO:0000008',
  'name': 'obsolete thioredoxin',
  'namespace': 'molecular_function',
  'alt_id': ['GO:0000013'],
  'def': '"OBSOLETE. A small disulfide-containing redox protein that serves as a general protein disulfide oxidoreductase. Interacts with a broad range of proteins by a redox mechanism, based on the reversible oxidation of 2 cysteine thiol groups to a disulfide, accompanied by the transfer of 2 electrons and 2 protons. The net result is t

In [77]:
def construct_nx_graph_from_obo_content(obo_terms: Iterable[dict], relationship_types_to_include: Container[str]) -> nx.DiGraph:
    """
    Construct a networkx graph from the OBO content.
    """
    def parse_is_a(is_a: str) -> tuple[str, str]:
        parent_id, parent_name = is_a.split(' ! ')
        return parent_id, parent_name
    
    def parse_relationship(relationship: str) -> tuple[str, str]:
        relationship_type, target = relationship.split(' ')
        target_id, target_name = target.split(' ! ')
        return relationship_type, target_id, target_name
    
    valid_relationship_types = set(relationship_types_to_include)

    graph = nx.DiGraph()
    # Add all non-obsolete terms to the graph
    for term in obo_terms:
        if 'is_obsolete' in term and term['is_obsolete'] == 'true':
            continue
        graph.add_node(term['id'], **term)

        # Add the "is_a" relationships in both parent and child directions
        if 'is_a' in term:
            for is_a in term['is_a']:
                parent_id, _ = parse_is_a(is_a)
                graph.add_edge(parent_id, term['id'], relationship_type='CHILD')
                graph.add_edge(term['id'], parent_id, relationship_type='PARENT')

        # Add the other relationships
        if 'relationship' in term:
            for relationship in term['relationship']:
                relationship_type, target_id, _ = parse_relationship(relationship)
                if relationship_type in valid_relationship_types:
                    graph.add_edge(term['id'], target_id, relationship_type=relationship_type)
    
        return graph

In [78]:
my_go_dag = construct_nx_graph_from_obo_content(content['Term'], set().union([type_def['id'][0] for type_def in content['Typedef']]))

In [79]:
my_go_dag

In [69]:
set().union([type_def['id'][0] for type_def in content['Typedef']])a

{'negatively_regulates',
 'part_of',
 'positively_regulates',
 'regulates',
 'term_tracker_item'}

In [61]:
for term in content['Term']:
    if 'relationship' in term:
        print(term)
        break

{'id': 'GO:0000015', 'name': 'phosphopyruvate hydratase complex', 'namespace': 'cellular_component', 'def': '"A multimeric enzyme complex, usually a dimer or an octamer, that catalyzes the conversion of 2-phospho-D-glycerate to phosphoenolpyruvate and water." [GOC:jl, ISBN:0198506732]', 'subset': 'goslim_metagenomics', 'synonym': '"enolase complex" EXACT []', 'is_a': 'GO:1902494 ! catalytic complex', 'relationship': 'part_of GO:0005829 ! cytosol'}


In [55]:
content['Term'][5:10]

[{'id': 'GO:0000007',
  'name': 'low-affinity zinc ion transmembrane transporter activity',
  'namespace': 'molecular_function',
  'def': '"Enables the transfer of a solute or solutes from one side of a membrane to the other according to the reaction: Zn2+ = Zn2+, probably powered by proton motive force. In low-affinity transport the transporter is able to bind the solute only if it is present at very high concentrations." [GOC:mtg_transport, ISBN:0815340729]',
  'is_a': 'GO:0005385 ! zinc ion transmembrane transporter activity'},
 {'id': 'GO:0000008',
  'name': 'obsolete thioredoxin',
  'namespace': 'molecular_function',
  'alt_id': 'GO:0000013',
  'def': '"OBSOLETE. A small disulfide-containing redox protein that serves as a general protein disulfide oxidoreductase. Interacts with a broad range of proteins by a redox mechanism, based on the reversible oxidation of 2 cysteine thiol groups to a disulfide, accompanied by the transfer of 2 electrons and 2 protons. The net result is the c

In [46]:
len(content['Term'])

44

In [37]:
max_lines = 1000000000
line_num = 0
atom_types = set([])
for line in obo_contents.split('\n'):
    line = line.strip()
    # print(line)
    if not line:
        continue
    if line[0] == '[' and line[-1] == ']':
        atom_types.add(line[1:-1])
    line_num += 1
    if line_num > max_lines:
        break

atom_types

{'Term', 'Typedef'}

In [28]:
atom_types

set()

In [30]:
len(go_file)

30992530

In [24]:
print(go_file[-1000:])

ynthesis" EXACT [GOC:obol]
is_a: GO:0018130 ! heterocycle biosynthetic process
is_a: GO:0034309 ! primary alcohol biosynthetic process
is_a: GO:0042181 ! ketone biosynthetic process
is_a: GO:0120255 ! olefinic compound biosynthetic process
is_a: GO:1901362 ! organic cyclic compound biosynthetic process
is_a: GO:2001316 ! kojic acid metabolic process

[Typedef]
id: negatively_regulates
name: negatively regulates
namespace: external
xref: RO:0002212
is_a: regulates ! regulates

[Typedef]
id: part_of
name: part of
namespace: external
xref: BFO:0000050
is_transitive: true

[Typedef]
id: positively_regulates
name: positively regulates
namespace: external
xref: RO:0002213
holds_over_chain: negatively_regulates negatively_regulates
is_a: regulates ! regulates

[Typedef]
id: regulates
name: regulates
namespace: external
xref: RO:0002211
is_transitive: true

[Typedef]
id: term_tracker_item
name: term tracker item
namespace: external
xref: IAO:0000233
is_metadata_tag: true
is_class_level: true



In [14]:
godag['GO:0000178']

GOTerm('GO:0000178'):
  id:GO:0000178
  item_id:GO:0000178
  name:exosome (RNase complex)
  namespace:cellular_component
  _parents: 1 items
    GO:1905354
  parents: 1 items
    GO:1905354	level-03	depth-03	exoribonuclease complex [cellular_component]
  children: 2 items
    GO:0000177	level-05	depth-05	cytoplasmic exosome (RNase complex) [cellular_component]
    GO:0000176	level-03	depth-05	nuclear exosome (RNase complex) [cellular_component]
  level:4
  depth:4
  is_obsolete:False
  alt_ids: 0 items

In [12]:
help(GODag)

Help on class GODag in module goatools.obo_parser:

class GODag(builtins.dict)
 |  GODag(
 |      obo_file: str = 'go-basic.obo',
 |      optional_attrs: Optional[set] = None,
 |      load_obsolete: bool = False,
 |      prt=<ipykernel.iostream.OutStream object at 0x10635bf40>
 |  )
 |
 |  Holds the GO DAG as a dict.
 |
 |  Method resolution order:
 |      GODag
 |      builtins.dict
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(
 |      self,
 |      obo_file: str = 'go-basic.obo',
 |      optional_attrs: Optional[set] = None,
 |      load_obsolete: bool = False,
 |      prt=<ipykernel.iostream.OutStream object at 0x10635bf40>
 |  )
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  draw_lineage(
 |      self,
 |      recs,
 |      nodecolor='mediumseagreen',
 |      edgecolor='lightslateblue',
 |      dpi=96,
 |      output='GO_lineage.png',
 |      engine='pygraphviz',
 |      gml=False,
 |      draw_parents=True,
 |      draw_childr